In [8]:
from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPool2D,Flatten,Dense,Dropout,Lambda
from keras.callbacks import TensorBoard
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import models

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
import numpy as np
#加载数据
X_R = np.load('raw_data.npy')
label_R = np.load('label.npy')
Y_R = label_R[:,1]
X_S = np.load('test_raw_data.npy')
label_S = np.load('test_label.npy')
Y_S = label_S[:,1]

In [50]:
x = np.load('raw_data.npy')
y = np.load('label.npy')
y = y[:,1:3]
print(y)

[[1 0]
 [1 0]
 [1 0]
 ...
 [0 2]
 [0 3]
 [0 3]]


In [3]:
X = np.vstack((X_R,X_S))
Y = np.append(Y_R,Y_S)

In [4]:
#数据归一化
mean_px = X.mean().astype(np.float32)
std_px = X.std().astype(np.float32)
X = (X-mean_px)/std_px

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=2,test_size = 0.2)

In [5]:
X_train.shape

(3194, 128, 128, 1)

In [5]:
def model():
    model=Sequential([
    Convolution2D(32,3,3,input_shape=(128,128,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Convolution2D(32,3,3,input_shape=(128,128,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
   ])
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    model.summary()
    return model

In [8]:
#训练保存模型
#model = model()
#model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test))
#model.save('cnn82.h5')
#model.save_weights('cnn82_weight.h5')

In [6]:
model2 = model()

f:\py\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
f:\py\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., activation="relu")`
  """



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1843264   
_________________________

In [86]:
model2.load_weights('cnn85_weight.h5')

In [87]:
dense1_layer_model = Model(inputs=model2.input,outputs=model2.layers[-2].output)

In [88]:
from sklearn.svm import SVC
import numpy as np

In [98]:
clf=SVC()

In [99]:
X_trian2 = dense1_layer_model.predict(X_train)
X_test2 = dense1_layer_model.predict(X_test)

In [100]:
clf.fit(X_trian2,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [101]:
y_p = clf.predict(X_test2)
acc = np.sum(y_p == y_test)/y_test.shape[0]
acc

0.8635794743429287

In [102]:
y_p2 = model2.predict_classes(X_test)
acc2 = np.sum(np.squeeze(y_p2) == y_test)/y_test.shape[0]
acc2

0.8560700876095119